In [25]:
import pandas as pd
import numpy as np

ratings = pd.read_csv(r"C:\Users\ASUS\Desktop\ml-100k\ml-100k\u.data",sep="\t",names=["user_id","item_id","rating","timestamp"])
utility_matrix=ratings.pivot_table(index="user_id", columns="item_id", values="rating")

user_means=utility_matrix.mean(axis=1)
centered_matrix=utility_matrix.sub(user_means,axis=0)

user1_id=1
item_id=508
user1=centered_matrix.loc[user1_id]
all_similarities=[]

for user in centered_matrix.index:
    if user==user1_id:
        continue
    user_centered=centered_matrix.loc[user]
    common_items=user1.notna()&user_centered.notna()
    if common_items.any():
        u1=user1[common_items].values.astype(float)
        u=user_centered[common_items].values.astype(float)
        dot_product=np.dot(u1, u)
        norm_u1=np.linalg.norm(u1)
        norm_u=np.linalg.norm(u)
        cosine=dot_product / (norm_u1*norm_u)if norm_u1*norm_u!=0 else 0.0
    else:
        cosine=0.0
    all_similarities.append((user,cosine))

top10=sorted(all_similarities,key=lambda x:x[1],reverse=True)[:10]
print("Top 10 most similar users (all users included):")
print([user for user,_ in top10])


Top 10 most similar users (all users included):
[418, 155, 341, 685, 812, 351, 811, 166, 810, 309]


In [26]:
numerator=0.0
denominator=0.0
for user, cosine in top10:
    rating_centered=centered_matrix.loc[user, item_id]
    numerator+=cosine*rating_centered
    denominator+=cosine

predicted_centered=numerator / denominator if denominator != 0 else 0
predicted_rating=user_means[user1_id] + predicted_centered

if np.isnan(predicted_rating):
    print("None of the 10 most similar users rated item 508")
else:
    print(f"Prediction of user 1's rating for item 508:{predicted_rating:.2f}")

None of the 10 most similar users rated item 508


In [27]:
top10_rated=[]
for user,cosine in sorted(all_similarities,key=lambda x:x[1],reverse=True):
    if not np.isnan(utility_matrix.loc[user,item_id]):
        top10_rated.append((user,cosine))
    if len(top10_rated)==10:
        break

print("Top 10 most similar users (rating item 508):")
print([user for user,_ in top10_rated])

Top 10 most similar users (rating item 508):
[266, 134, 413, 905, 199, 32, 66, 526, 69, 120]


In [28]:
numerator=0.0
denominator=0.0
for user,cosine in top10_rated:
    rating_centered=centered_matrix.loc[user,item_id]
    numerator+=cosine*rating_centered
    denominator+=cosine

predicted_centered=numerator/denominator if denominator!=0 else 0
predicted_rating=user_means[user1_id]+predicted_centered

print(f"Prediction of user 1's rating for item 508:{predicted_rating:.2f}")

Prediction of user 1's rating for item 508:3.89
